# Распределение встреч между представителями

Данный ноутбук содержит в себе три крупных блока:
* Подготовка данных
* Эвристика
* Сохранение результат

Ожидается, что доработки будут только во второй части.

Метрика качества, которую требуется оптимизировать, реализована в функции `compute_all_metrics`.
Финальная матрика состоит из двух слагаемых:
* Суммарное время, потраченное представителями на дорогу
* Сумма квадратов опозданий на встречи с весом 0.02

Модель, используемая в данной задачу следующая:
* Представитель начинает работу в офисе и отправляется из офиса в начало своего рабочего дня
* В случае, если представитель приезжает на встречу раньше запланированного времени, он ожидает на одном месте начала встречи

In [1]:
import pandas as pd
import random
import numpy as np
import time
import copy

## Подготовка данных

Данные выглядят следующим образом:

Информация о встречах находится в таблице **F_ACT.csv**
* `ID` - уникальный id встречи
* `station_id` - id станции метро
* `SCENARIO_ID` - id сценария встречи. Необходим для определения длительности встречи проверки соответсвия навыкам представителя
* `READY_TIME` - начало интервала встречи
* `DUE_TIME` - окончание интервала встречи
* `from_office` - время, необходимое на дорогу от офиса

Длительность встреч **F_SCENARIO.csv**
* `ID` - id сценария
* `DURATION` - длительность встречи с данным сценарием

Матрица расстояний **F_DISTANCE.csv**
* `id1` - id первой встречи
* `id2` - id второй встречи
* `dist` - время, необходимое на дорогу от первой до второй встречи

Время работы представителей **F_REAL_WORK.csv**
* `agent_id` - id представителя
* `SLOT_START` - начало рабочего дня представителя
* `SLOT_END` - окончание рабочего дня представителя

Скиллы представителей **F_SKILLS.csv**
* `agent_id` - id представителя
* `SCENARIO_ID` - id сценария, которым владеет данный представитель


In [2]:
agents = pd.read_csv('data/F_REAL_WORK.csv', sep=';', index_col='agent_id').to_dict(orient='index')
i = 0
print('len(agents) =', len(agents))
for id, agent in agents.items():
    print(id, agent)
    i += 1
    if i == 5:
        break

len(agents) = 74
6830 {'SLOT_START': 28800, 'SLOT_END': 61200}
465 {'SLOT_START': 28800, 'SLOT_END': 61200}
400 {'SLOT_START': 28800, 'SLOT_END': 61200}
3720 {'SLOT_START': 28800, 'SLOT_END': 61200}
673 {'SLOT_START': 28800, 'SLOT_END': 61200}


In [3]:
activities = pd.read_csv('data/F_ACT.csv', sep=';', index_col='ID').to_dict(orient='index')
i = 0
print('len(activities) =', len(activities))
for id, act in activities.items():
    print(id, act)
    i += 1
    if i == 5:
        break

len(activities) = 1154
3385646 {'station_id': 1, 'SCENARIO_ID': 30, 'READY_TIME': 43200, 'DUE_TIME': 50400, 'from_office': 3080}
3385152 {'station_id': 1, 'SCENARIO_ID': 34, 'READY_TIME': 36000, 'DUE_TIME': 43200, 'from_office': 3080}
3385097 {'station_id': 1, 'SCENARIO_ID': 30, 'READY_TIME': 54000, 'DUE_TIME': 61200, 'from_office': 3080}
3385201 {'station_id': 1, 'SCENARIO_ID': 33, 'READY_TIME': 54000, 'DUE_TIME': 61200, 'from_office': 3080}
3385084 {'station_id': 1, 'SCENARIO_ID': 30, 'READY_TIME': 64800, 'DUE_TIME': 72000, 'from_office': 3080}


In [4]:
skills = pd.read_csv('data/F_SKILLS.csv', sep=';')
skills_set = set()
for i in skills.itertuples():
    skills_set.add((i[1], i[2]))
i = 0
print('len(skills_set) =', len(skills_set))
for skill in skills_set:
    print(skill)
    i += 1
    if i == 5:
        break
skills.head()

len(skills_set) = 1427
(529, 69)
(2024, 43)
(443, 32)
(183, 34)
(449, 74)


,agent_id,SCENARIO_ID
0,183,30
1,183,31
2,183,32
3,183,33
4,183,34


In [5]:
scenarios = pd.read_csv('data/F_SCENARIO.csv', sep=';', index_col='ID').to_dict(orient='index')
i = 0
print('len(scenarios) =', len(scenarios))
for id, scenario in scenarios.items():
    print(id, scenario)
    i += 1
    if i == 5:
        break

len(scenarios) = 60
181 {'DURATION': 10}
201 {'DURATION': 10}
30 {'DURATION': 20}
31 {'DURATION': 10}
32 {'DURATION': 20}


In [6]:
distance = pd.read_csv('data/F_DISTANCE.csv', sep=';', index_col=['id1', 'id2'])
distance.head(5)

dist
id1     id2          
3385891 3386028  3730
        3385895  3730
3385788 3386028  3730
3386106 3386134  3540
3386108 3386134  3540

In [7]:
id_by_act = {act: i for i, act in enumerate(list(activities))}
i = 0
print('len(id_by_act) =', len(id_by_act))
for id, act_ in id_by_act.items():
    print(id, act_)
    i += 1
    if i == 5:
        break

len(id_by_act) = 1154
3385646 0
3385152 1
3385097 2
3385201 3
3385084 4


In [8]:
distance_by_id = np.zeros((len(activities), len(activities)))

In [9]:
%%time
for (i, j), row in distance.iterrows():
    id_i = id_by_act[i]
    id_j = id_by_act[j]
    distance_by_id[id_i][id_j] = row['dist']
    distance_by_id[id_j][id_i] = row['dist']
print(distance_by_id)

[[    0.  1558.  1800. ...,  1800.  1800.  1800.]
 [ 1558.     0.  1800. ...,  1800.  1800.  1800.]
 [ 1800.  1800.     0. ...,  1800.  1800.  1800.]
 ..., 
 [ 1800.  1800.  1800. ...,     0.  1800.  1800.]
 [ 1800.  1800.  1800. ...,  1800.     0.  1800.]
 [ 1800.  1800.  1800. ...,  1800.  1800.     0.]]
Wall time: 1min 17s


#### Определение доп.функций

In [10]:
def get_end_by_act(act):
    return activities[act]['DUE_TIME']

In [11]:
def get_distance_between_act(from_act, to_act):
    return distance_by_id[id_by_act[from_act]][id_by_act[to_act]]

In [12]:
def agent_have_skill(agent, skill):
    return (agent, skill) in skills_set

In [13]:
def sort_solution(solution):
    for agent in solution:
        solution[agent].sort(key=get_end_by_act)
    #return solution

In [14]:
def create_empty_solution(agents):
    result = dict()
    score = 0
    for agent in agents:
        result.update({agent: []})
    return result

In [15]:
def get_random_solution():
    solution = create_empty_solution(agents)
    for act in activities:
        skill = activities[act]['SCENARIO_ID']
        random_agent = random.choice(list(skills[skills.SCENARIO_ID == skill]['agent_id']))
        solution[random_agent].append(act)
    sort_solution(solution)
    return solution

#### Подсчет метрик

In [16]:
def compute_total_travel_time(solution):
    total_time = 0
    for agent in solution:
        if len(solution[agent]) == 0:
            continue
        first_act = solution[agent][0]
        total_time += activities[first_act]['from_office']
        for i in range(len(solution[agent]) - 1):
            total_time += get_distance_between_act(solution[agent][i], solution[agent][i+1])
    return total_time

In [17]:
def compute_start_times(solution):
    activity_start = dict()
    for agent in solution:
        if len(solution[agent]) == 0:
            continue
        first_act = solution[agent][0]
        activity_start[first_act] = max(activities[first_act]['READY_TIME'],
                                        agents[agent]['SLOT_START'] + activities[first_act]['from_office'])
        for i in range(1, len(solution[agent])):
            current_act = solution[agent][i]
            prev_act = solution[agent][i - 1]
            dist = get_distance_between_act(prev_act, current_act)
            skill_prev_act = activities[prev_act]['SCENARIO_ID']
            activity_start[current_act] = max(activities[current_act]['READY_TIME'], 
                                              activity_start[prev_act] + scenarios[skill_prev_act]['DURATION'] * 60 + dist)
    return activity_start

In [18]:
def compute_wait_time(solution):
    activity_start = compute_start_times(solution)
    total_wait = 0
    for act, start_act in activity_start.items():
        local_wait = max(0, start_act - activities[act]['DUE_TIME'])
        total_wait += local_wait**2
    return total_wait

In [19]:
def compute_all_metrics(solution):
    total_travel = compute_total_travel_time(solution)
    total_wait = compute_wait_time(solution)
    return total_travel + 0.02 * total_wait

### Эвристика

Дальнейшую часть кода нужно самостоятельно дополнить.
Ожидается, что будут написаны дополнительные способы мутации одного решения, изменение способа отбора решений в одном поколении.

Ограничений на эвристику нет, можно делать все что угодно.

#### Случайные изменения

In [20]:
def merge_greedly_by_time(solution, k):
    candidates = random.choices(list(solution), k=k)
    cand_acts = []
    for e in candidates:
        cand_acts.extend(solution[e])
        solution[e] = []
    cand_acts.sort(key=get_end_by_act)
    for act in cand_acts:
        best_candidate = -1
        best_dist = -1
        skill = activities[act]['SCENARIO_ID']
        for candidate in candidates:
            if not agent_have_skill(candidate, skill):
                continue
            if len(solution[candidate]) == 0:
                p_dist = activities[act]['from_office']
            else:
                p_dist = get_distance_between_act(solution[candidate][-1], act)
            if ((p_dist < best_dist) or (best_dist == -1)):
                best_dist = p_dist
                best_candidate = candidate
        assert best_candidate != -1
        solution[best_candidate].append(act)
    return solution

In [21]:
def random_transfer(solution):
    while True:
        agent = random.choice(list(solution))
        if len(solution[agent]) != 0:
            break
    act = random.choice(solution[agent])
    skill = activities[act]['SCENARIO_ID']
    solution[agent].remove(act)
    new_agent = random.choice(skills[skills.SCENARIO_ID == skill].agent_id.values)
    solution[new_agent].append(act)
    sort_solution(solution)
    #print(solution)
    return solution

#### Обмен встречами между наиболее загруженными агентами и наименее загруженными агентами 

In [22]:
def get_max_charged_cand(solution, number):
    '''Возвращает список наиболее загруженных агентов, то есть метрика качества которых наибольшая.'''
    candidates = []
    sol = copy.deepcopy(solution)
    for _ in range(number):
        max_charged_cand = list(sol.keys())[0]
        max_metric = compute_all_metrics({max_charged_cand: sol[max_charged_cand]})
        for agent in sol:
            metric = compute_all_metrics({agent: sol[agent]})
            if metric > max_metric:
                max_charged_cand = agent
                max_metric = metric
        candidates.append(max_charged_cand)
        sol.pop(max_charged_cand)
    return candidates

In [23]:
def get_min_charged_cand(solution, number):
    '''Возвращает список наименее загруженных агентов, то есть метрика качества которых наименьшая.'''
    candidates = []
    sol = copy.deepcopy(solution)
    for _ in range(number):
        min_charged_cand = list(sol.keys())[0]
        min_metric = compute_all_metrics({min_charged_cand: sol[min_charged_cand]})
        for agent in sol:
            metric = compute_all_metrics({agent: sol[agent]})
            if metric < min_metric:
                min_charged_cand = agent
                min_metric = metric
        candidates.append(min_charged_cand)
        sol.pop(min_charged_cand)
    return candidates

In [24]:
def max_charged_crossover_min_charged(solution, k):
    '''
    Возвращает решение, в котором встречи распределены между k наиболее загруженными и k/3 наименее
    загруженными агентами.
    '''
    candidates = get_max_charged_cand(solution, k - int(k / 3))
    candidates.extend(get_min_charged_cand(solution, int(k / 3)))
    cand_acts = []
    for e in candidates:
        cand_acts.extend(solution[e])
        solution[e] = []
    cand_acts.sort(key=get_end_by_act)
    for act in cand_acts:
        best_candidate = -1
        best_dist = -1
        skill = activities[act]['SCENARIO_ID']
        for candidate in candidates:
            if not agent_have_skill(candidate, skill):
                continue
            if len(solution[candidate]) == 0:
                p_dist = activities[act]['from_office']
            else:
                p_dist = get_distance_between_act(solution[candidate][-1], act)
            if ((p_dist < best_dist) or (best_dist == -1)):
                best_dist = p_dist
                best_candidate = candidate
        assert best_candidate != -1
        solution[best_candidate].append(act)
    return solution

In [25]:
def worst_act_transfer(solution):
    '''
    Возвращает решение, в котором для наиболее загруженного агента была удалена встреча,
    которая вносила наибольший вклад в его метрику качества. А затем она была добавлена агенту, 
    для которого вклад этой встречи в его метрику качества наименьший.
    '''
    max_agent = get_max_charged_cand(solution, 1)[0]
    max_metric = -1
    max_act = -1
    for act in solution[max_agent]:        
        acts = copy.deepcopy(solution[max_agent])
        acts.remove(act)
        metric = compute_all_metrics({max_agent: acts})
        if metric > max_metric:
            max_act = act
            max_metric = metric
    skill = activities[max_act]['SCENARIO_ID']
    solution[max_agent].remove(max_act)
    agents_with_skill = skills[skills.SCENARIO_ID == skill].agent_id.values
    min_metric = max_metric
    min_agent = -1
    for agent in agents_with_skill:
        acts = copy.deepcopy(solution[agent])
        acts.append(max_act)
        acts.sort(key=get_end_by_act)
        metric = compute_all_metrics({agent: acts})
        if metric < min_metric:
            min_metric = metric
            min_agent = agent
    solution[min_agent].append(max_act)
    return solution

In [26]:
def merge_max_and_min_charged(solution, k):
    '''
    Возвращает решение, для которого между k/3 наиболее загруженными  и 2k/3 наименее загруженными
    агентами были распределены встречи.
    '''
    candidates = get_max_charged_cand(solution, int(k / 3))
    candidates.extend(get_min_charged_cand(solution, k - int(k / 3)))
    cand_acts = []
    for e in candidates:
        cand_acts.extend(solution[e])
        solution[e] = []
    cand_acts.sort(key=get_end_by_act)
    for act in cand_acts:
        best_candidate = -1
        best_dist = -1
        skill = activities[act]['SCENARIO_ID']
        for candidate in candidates:
            if not agent_have_skill(candidate, skill):
                continue
            if len(solution[candidate]) == 0:
                p_dist = activities[act]['from_office']
            else:
                p_dist = get_distance_between_act(solution[candidate][-1], act)
            if ((p_dist < best_dist) or (best_dist == -1)):
                best_dist = p_dist
                best_candidate = candidate
        assert best_candidate != -1
        solution[best_candidate].append(act)
    return solution

### Кроссовер

Для применения кроссовера между двумя решениями поколения было принято решение изменить структуру решений, которая позволила бы при скрещивании не потерять некоторые встречи. А именно - каждой встрече при помощи словаря поставим в соответствие агента. Так как на 1 встречу приезжает 1 агент, при таком представлении информация о встречах не потеряется. Те агенты, которые не попадут в такое представление решения, просто не имеют встреч. Это будет учитываться при возврате к исходной структуре решения.

In [27]:
def get_solution_key_act(solution):
    '''
    Возвращает подаваемое на вход решение, в виде словаря, в котором ключом являются
    встречи, а значением - агенты.    
    '''
    changed_solution = dict()
    for agent, actions in solution.items():
        for act in actions:
            changed_solution[act] = agent
    return changed_solution

In [28]:
def get_solution_key_agent(solution):
    '''
    Возвращает подаваемое на вход инвертированное решение, в виде словаря, в котором ключом являются
    агенты, а значением - список встреч соответсвующего агента.    
    '''
    changed_solution = dict()
    for act, agent in solution.items():
        if agent not in changed_solution:
            changed_solution[agent] = []
        changed_solution[agent].append(act)
    sort_solution(changed_solution)
    for agent in agents:
        if agent not in changed_solution:
            changed_solution[agent] = []
    return changed_solution

#### One Point Crossover

Встречи упорядочим по времени их окончания и соединим первую половину одного решения со второй половиной второго.

In [29]:
def one_point_crossover(sol1, sol2):
    parent1 = get_solution_key_act(sol1)
    parent2 = get_solution_key_act(sol2)
    actions = list(parent1.keys())
    actions.sort(key=get_end_by_act)
    child1 = dict()
    child2 = dict()
    for i in range(int(len(actions) / 2)):
        child1[actions[i]] = parent1[actions[i]]
        child2[actions[i]] = parent2[actions[i]]
    for i in range(int(len(actions) / 2), len(actions)):
        child1[actions[i]] = parent2[actions[i]]
        child2[actions[i]] = parent1[actions[i]]
    solution1 = get_solution_key_agent(child1)
    solution2 = get_solution_key_agent(child2)
    return [solution1, solution2]

Выберем произвольную точку, относительно которой буудем производить обмен между решениями. 

In [30]:
def one_random_point_crossover(sol1, sol2):
    parent1 = get_solution_key_act(sol1)
    parent2 = get_solution_key_act(sol2)
    actions = list(parent1.keys())
    actions.sort(key=get_end_by_act)
    child1 = dict()
    child2 = dict()
    point = random.randint(1, len(actions) - 1)
    for i in range(point):
        child1[actions[i]] = parent1[actions[i]]
        child2[actions[i]] = parent2[actions[i]]
    for i in range(point, len(actions)):
        child1[actions[i]] = parent2[actions[i]]
        child2[actions[i]] = parent1[actions[i]]
    solution1 = get_solution_key_agent(child1)
    solution2 = get_solution_key_agent(child2)
    return [solution1, solution2]

Выберем заданное число произвольных точек, которые разбиывают два решения на несколько частей. Первых потомок составляется из нечетных частей первого родителя и четных частей второго родителя. Аналогично второй потомок составляется из четных частей первого родителя и нечетных частей второго родителя.

In [31]:
def multi_random_point_crossover(sol1, sol2, n_points):
    parent1 = get_solution_key_act(sol1)
    parent2 = get_solution_key_act(sol2)
    actions = list(parent1.keys())
    actions.sort(key=get_end_by_act)
    child1 = dict()
    child2 = dict()
    points = [0]
    for i in range(n_points):
        points.append(random.choice(list(range(1, len(actions) - 1))))
    points = list(set(points))
    points.sort()
    points.append(None) 
    prev_point = 0
    for n in range(len(points) - 1):
        if n % 2 == 0:
            if points[n + 1] != None:
                for i in range(points[n], points[n + 1]):
                    child1[actions[i]] = parent1[actions[i]]
                    child2[actions[i]] = parent2[actions[i]]
            else:
                for i in range(points[n], len(actions)):
                    child1[actions[i]] = parent1[actions[i]]
                    child2[actions[i]] = parent2[actions[i]]                
        else:            
            if points[n + 1] != None:
                for i in range(points[n], points[n + 1]):
                    child1[actions[i]] = parent2[actions[i]]
                    child2[actions[i]] = parent1[actions[i]]
            else:
                for i in range(points[n], len(actions)):
                    child1[actions[i]] = parent2[actions[i]]
                    child2[actions[i]] = parent1[actions[i]]  
    solution1 = get_solution_key_agent(child1)
    solution2 = get_solution_key_agent(child2)
    return [solution1, solution2]

#### Создание поколения

In [32]:
generation_size = 20
initial_generation = []
for _ in range(generation_size):
    one_solution = get_random_solution()
    initial_generation.append((compute_all_metrics(one_solution), one_solution))

In [33]:
initial_generation.sort()

In [34]:
initial_generation[0][0]

15375324356.58

#### Получение следующего поколения

In [35]:
labeled_res = dict()

In [36]:
labeled_res[0] = initial_generation[0][0]
best_generation = initial_generation[0][1]

In [37]:
generation = copy.deepcopy(initial_generation)
for i in range(50):
    new_generation = copy.deepcopy(generation)
    for g in generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        new_generation.append((compute_all_metrics(new_solution), new_solution))
    new_generation.sort(key=lambda t: t[0])
    generation = new_generation[:generation_size]
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
merge_greedly_by_time_metric = generation[0][0]
print('best solution', merge_greedly_by_time_metric)
labeled_res[1] = merge_greedly_by_time_metric

[12941990769.76, 13170566889.82, 13657198968.52, 13820805738.68, 14010543556.040001]
[6916930106.46, 7071746882.3199997, 7173679562.8800001, 7746874764.8800001, 7849098729.04]
[6046631884.6000004, 6282273917.5600004, 6528399591.1800003, 6540691173.2799997, 6615485388.6000004]
[5243472266.4200001, 5267597167.1999998, 5291103254.5200005, 5317599654.6800003, 5319963637.2399998]
[4803355451.4000006, 5021693349.3599997, 5047345961.1999998, 5085689833.4200001, 5243472266.4200001]
best solution 4803355451.4


In [38]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
4803355451.4
4803355451.4


In [39]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 1:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
result became better = True


Для каждого решения из поколения на каждой итерации произведем следующий отбор лучшего решения: распределим встречи между k наиболее загруженными и k/3 наименее загруженными агентами.

In [40]:
generation = copy.deepcopy(initial_generation)
for i in range(50):
    new_generation = copy.deepcopy(generation)
    for g in generation:
        new_solution = max_charged_crossover_min_charged(copy.deepcopy(g[1]), 20)
        new_generation.append((compute_all_metrics(new_solution), new_solution))
    new_generation.sort(key=lambda t: t[0])
    generation = new_generation[:generation_size]
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
max_charged_crossover_min_charged_metric = generation[0][0]
print('best solution', max_charged_crossover_min_charged_metric)
labeled_res[2] = max_charged_crossover_min_charged_metric

[8643682071.8600006, 10380649679.219999, 10678843639.800001, 11128443586.0, 11642768743.16]
[8643682071.8600006, 8643682071.8600006, 10380649679.219999, 10380649679.219999, 10602334403.16]
[8643682071.8600006, 8643682071.8600006, 10380649679.219999, 10380649679.219999, 10602334403.16]
[8643682071.8600006, 8643682071.8600006, 10380649679.219999, 10380649679.219999, 10602334403.16]
[8643682071.8600006, 8643682071.8600006, 10380649679.219999, 10380649679.219999, 10602334403.16]
best solution 8643682071.86


In [41]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 2:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
result became better = False


In [42]:
def get_random_mutation(generat):
    '''Возвращает решения, над которыми была произведена произвольная мутация.'''
    new_generat = []
    for i in range(len(generat)):
        solution = generat[i][1]
        new_solution = random_transfer(solution)
        new_generat.append((compute_all_metrics(new_solution), new_solution))
    new_generat.sort(key=lambda t: t[0])
    #print('best solution', new_generat[0][0])
    return new_generat

Определим ниже распределяющую мутацию

In [43]:
def get_distributing_mutation(generat):
    '''
    Возвращает решения, для каждого из которых для наиболее загруженного агента была удалена встреча, 
    вносящая наибольший вклад в его метрику качества. А затем она была добавлена агенту, для которого 
    вклад этой встречи в его метрику качества наименьший. Назовем такую мутацию распределяющей.
    '''
    new_generat = []
    for i in range(len(generat)):
        solution = generat[i][1]
        new_solution = worst_act_transfer(solution)
        new_generat.append((compute_all_metrics(new_solution), new_solution))
    new_generat.sort(key=lambda t: t[0])
    #print('best solution', new_generat[0][0])
    return new_generat

Добавим 3 типа мутации в каждый этап формирования поколения. Мутации будут производится до отбрасывания худших решений, а добавляться после него для разнообразия генофонда.

In [44]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    childs = []
    for g in generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 4) - 9]
    generation.extend(childs[:(int)(generation_size / 4)])
    for sol in mutated_solutions:
        generation.append(sol)
        
    generation.sort(key=lambda t: t[0])    
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
mutations_metric = generation[0][0]
print('best solution', mutations_metric)
labeled_res[3] = mutations_metric

[13138600373.200001, 13378462464.9, 13535631158.34, 13540363953.780001, 13681727816.040001]
[5932927687.7600002, 5967986168.6000004, 6014757272.1599998, 6015321564.2799997, 6015321564.2799997]
[5097847070.4400005, 5133260907.96, 5139533150.4400005, 5142696820.4400005, 5146152689.8000002]
[4005818066.8000002, 4020125083.0, 4043608109.3000002, 4050107454.8600001, 4051458513.0]
[3700088517.6399999, 3721459585.6399999, 3749131251.8200002, 3752265472.9000001, 3760295222.9400001]
best solution 3208763123.02


In [45]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
3208763123.02
3208763123.02


In [46]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 3:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
result became better = True


Добавим кроссовер двух решений с одной точкой

In [47]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    childs = []
    for g in generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution))
        
    #скрещиваем лучшие 6 решений между собой произвольным образом
    for _ in range(6):        
        new_solutions = one_random_point_crossover(random.choice(new_generation[:6])[1],
                                           random.choice(new_generation[:6])[1])
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution))
    
    #скрещиваем лучшие 6 решений с худшими 6 решениями произвольным образом
    for _ in range(6):
        new_solutions = one_random_point_crossover(random.choice(new_generation[:6])[1],
                                           random.choice(new_generation[-6:])[1])
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 4) - 6]
    generation.extend(childs[:(int)(generation_size / 4)])
    for sol in mutated_solutions:
        generation.append(sol)
                
    generation.sort(key=lambda t: t[0])
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
one_point_crossover_metric = generation[0][0]
print('best solution', one_point_crossover_metric)
labeled_res[4] = one_point_crossover_metric

[13183808440.68, 13411983551.380001, 13423721047.16, 13564159387.4, 13939840463.040001]
[6185469099.7600002, 6326947567.8599997, 6398241927.6400003, 6398343186.1000004, 6492181834.3800001]
[4987204595.9800005, 5016965041.5799999, 5036196796.4800005, 5036196796.4800005, 5036196796.4800005]
[4248305172.6199999, 4256811202.48, 4268734778.48, 4272633950.6199999, 4272633950.6199999]
[3954214539.4000001, 3958803724.8600001, 3978757281.8200002, 3978757281.8200002, 3978757381.8200002]
best solution 3519173551.1


In [48]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
3519173551.1
3519173551.1


In [49]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 4:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
4 3519173551.1
result became better = False


Добавим кроссовер двух решений с несколькими точками 

In [50]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    childs = []
    for g in generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution))
        
    #скрещиваем лучшие 6 решений между собой произвольным образом
    for _ in range(6):        
        #print(len(random.choice(generation[:6])[1]))
        new_solutions = multi_random_point_crossover(random.choice(new_generation[:6])[1],
                                                     random.choice(new_generation[:6])[1], 
                                                     4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution))
    
    #скрещиваем лучшие 6 решений с худшими 6 решениями произвольным образом
    for _ in range(6):
        #print(len(random.choice(generation[:6])[1]))
        new_solutions = multi_random_point_crossover(random.choice(new_generation[:6])[1],
                                                     random.choice(new_generation[-6:])[1],
                                                     4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 4) - 6]
    generation.extend(childs[:(int)(generation_size / 4)])
    for sol in mutated_solutions:
        generation.append(sol)
                
    generation.sort(key=lambda t: t[0])
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
multi_random_point_crossover_metric = generation[0][0]
print('best solution', multi_random_point_crossover_metric)
labeled_res[5] = multi_random_point_crossover_metric

[13710365971.16, 13863941473.26, 14137141819.040001, 14312166369.66, 14623556931.120001]
[6876043485.4400005, 6902348515.9800005, 7159305413.0600004, 7161615359.1400003, 7515131028.5799999]
[4861063469.4200001, 4861063469.4200001, 4881077170.1999998, 4881077170.1999998, 4881077170.1999998]
[4351581374.96, 4372660826.0, 4372902588.6400003, 4377688125.5200005, 4391714794.3400002]
[4140756051.2200003, 4158867451.2400002, 4158867451.2400002, 4168580163.2400002, 4168580163.2400002]
best solution 3978039006.72


In [51]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
3978039006.72
3978039006.72


In [52]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 5:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
4 3519173551.1
5 3978039006.72
result became better = False


Изменим выбор родителей для скрещивания и добавим распределение между наиболее загруженными и наименее загруженными агентами.

In [53]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    for j in range(len(new_generation)):
        if (len(new_generation[j]) == 2):
            new_generation[j] = new_generation[j] + (0,)
        else:
            new_generation[j] = new_generation[j][:2] + (0,)
    new_generation.sort(key=lambda t: t[0]) 
    childs = []
    for g in generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 1))
        
    for g in generation:
        new_solution = merge_max_and_min_charged(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 2))    
    
    #скрещиваем лучшие 6 решений между собой произвольным образом
    for _ in range(6):        
        #print(len(random.choice(generation[:6])[1]))
        new_solutions = multi_random_point_crossover(random.choice(new_generation[:6])[1],
                                                     random.choice(new_generation[:6])[1], 
                                                     4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 3))
    childs.sort(key=lambda t: t[0]) 
        
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными решениями
    for _ in range(6):      
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        best_solution2 = max(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, best_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 4))
    
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными худшими решениями
    for _ in range(6):
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        worst_solution2 = min(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, worst_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 5))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 4) - 6]    
    
    generation.extend(childs[:(int)(generation_size / 4)])
    for sol in mutated_solutions:
        generation.append(sol + (6,))
                
    generation.sort(key=lambda t: t[0])
    print([x[2] for x in generation])
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
child_selection_metric1 = generation[0][0]
print('best solution', child_selection_metric1)
labeled_res[6] = child_selection_metric1

[2, 2, 2, 2, 1, 6, 0, 6, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[12194629510.02, 12270014731.300001, 12465030820.1, 12521524297.559999, 12950130025.800001]
[2, 1, 1, 1, 2, 6, 0, 6, 6, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[2, 1, 1, 6, 6, 0, 2, 1, 6, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[1, 6, 0, 2, 2, 6, 2, 6, 6, 3, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[1, 6, 1, 0, 6, 1, 2, 6, 3, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[2, 6, 0, 6, 3, 3, 3, 1, 6, 0, 6, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[1, 1, 1, 5, 1, 6, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[2, 6, 6, 0, 6, 6, 0, 3, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[2, 1, 6, 1, 6, 0, 2, 1, 6, 0, 0, 6, 0, 0, 0, 0, 0, 0, 6, 6]
[0, 6, 6, 6, 2, 0, 2, 1, 6, 0, 0, 0, 3, 0, 0, 2, 0, 0, 6, 6]
[2, 1, 6, 3, 0, 0, 0, 3, 3, 6, 0, 6, 0, 0, 6, 0, 0, 0, 6, 6]
[5396489456.8000002, 5584080060.9000006, 5668392208.1000004, 5673307660.3800001, 5679651419.1000004]
[6, 6, 0, 2, 2, 6, 0, 1, 3, 0, 0, 3, 0, 0, 0, 0, 0, 6, 6, 6]
[6, 6, 6, 0, 3, 3, 3, 3, 3, 6, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6]
[6, 3, 5, 6, 6

In [54]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
3197990013.22
3197990013.22


In [55]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 6:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
4 3519173551.1
5 3978039006.72
6 3197990013.22
result became better = True


Изменим отбор потомков. В новую популяцию войдут потомки с наилучшими значениями метрики, но при этом по возможности каждая метрика встретится хотя бы один раз. То есть если уникальных метрик больше возможного количества потомков, то среди решений с одинаковой метрикой будет выбрано одно из них. Если уникальных метрик меньше возможного количесвто потомков, то сначала буду добавлены по 1 решению с каждым уникальным значеним метрики, а затем добавятся оставшиеся решения с лучшей метрикой.

In [56]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    for j in range(len(new_generation)):
        if (len(new_generation[j]) == 2):
            new_generation[j] = new_generation[j] + (0,)
        else:
            new_generation[j] = new_generation[j][:2] + (0,)
    new_generation.sort(key=lambda t: t[0]) 
    childs = []
    for g in new_generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 1))
    for g in new_generation:
        new_solution = merge_max_and_min_charged(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 2))    
        
    childs = childs[:generation_size]
    
    #скрещиваем лучшие 6 решений между собой произвольным образом
    for _ in range(6):        
        #print(len(random.choice(generation[:6])[1]))
        new_solutions = multi_random_point_crossover(random.choice(new_generation[:6])[1],
                                                     random.choice(new_generation[:6])[1], 
                                                     4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 3))
    childs.sort(key=lambda t: t[0]) 
        
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными решениями
    for _ in range(6):      
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        best_solution2 = max(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, best_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 4))
    
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными худшими решениями
    for _ in range(6):
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        worst_solution2 = min(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, worst_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 5))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 4) - 6]    
    
    #добавим в новую популяцию хотя бы 1 потомка с уникальным значением метрики
    unique_fv = []
    unique_fv_index = []
    for k in range(len(childs)):
        if (childs[k][0] not in unique_fv) & (len(unique_fv) < (int)(generation_size / 4)):
            generation.append(childs[k])
            unique_fv.append(childs[k][0])
            unique_fv_index.append(k)
    while len(unique_fv) < (int)(generation_size / 4):
        print('not unique metriic')
        for k in range(len(childs)):
            if k not in unique_fv_index:
                generation.append(childs[k])
                unique_fv.append(childs[k][0])
                
    for sol in mutated_solutions:
        generation.append(sol + (6,))
                
    generation.sort(key=lambda t: t[0])
    
    childs_count = np.array([[0, 0, 0, 0, 0, 0, 0]])
    for sol in generation:
        if sol[2] == 0:
            childs_count[0][0] += 1
        elif sol[2] == 1:
            childs_count[0][1] += 1
        elif sol[2] == 2:
            childs_count[0][2]+=1
        elif sol[2] == 3:
            childs_count[0][3]+=1
        elif sol[2] == 4:
            childs_count[0][4]+=1
        elif sol[2] == 5:
            childs_count[0][5]+=1
        else:
            childs_count[0][6]+=1
    tb = pd.DataFrame(childs_count, columns = ['0', '1', '2', '3', '4', '5', '6'])
   # display(tb)
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
child_selection_metric1 = generation[0][0]
print('best solution', child_selection_metric1)
labeled_res[7] = child_selection_metric1
tb.head(20)

[13788236300.940001, 13890530794.800001, 14910378771.860001, 14943345872.32, 15030740428.16]
[6628127710.6800003, 6686062438.0600004, 6686062438.0600004, 6692615640.6599998, 6727588882.1599998]
[4550085964.6999998, 4550085964.6999998, 4567702949.8999996, 4572656928.6999998, 4590240424.6999998]
[3880978553.54, 3887497527.1399999, 3888192821.1399999, 4075822584.5599999, 4079201157.8200002]
[3673606403.7000003, 3673606510.7000003, 3686381841.0999999, 3686430960.46, 3686660424.46]
best solution 3274088830.24


,0,1,2,3,4,5,6
0,9,0,0,5,0,0,6


In [57]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
3274088830.24
3274088830.24


In [58]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 7:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
4 3519173551.1
5 3978039006.72
6 3197990013.22
7 3274088830.24
result became better = False


Введем селекцию потомков, вероятность которой обратно пропоциональна его метрике.

In [59]:
def roulete_wheel_selection(generation, n):
    metric = []
    #generation.sort(reverse=True, key=lambda t: t[0])
    for g in generation:
        metric.append(g[0])
    fv = [1 / x for x in metric]
    fv_sum = sum(fv)
    rel_fv = [x / fv_sum for x in fv]
    probs = [sum(rel_fv[:i+1]) for i in range(len(rel_fv))]
    # Draw new population
    sols = []
    #print(probs)
    for _ in range(n):
        r = random.random()
        for (i, individual) in enumerate(generation):
            if r <= probs[i]:
                sols.append(individual)
                break
    return sols

Введем селекцию потомков, вероятность которой пропорциональна его рангу при упорядочивании по возрастанию метрики.

In [60]:
def roulete_wheel_selection1(generation, n):
    metric = []
    #generation.sort(reverse=True, key=lambda t: t[0])
    for g in generation:
        metric.append(g[0])
    unique_metric_count = len(set(metric)) +1
    prev_m = -1
    rang = []
    for i in range(len(metric)):
        #print(unique_metric_count)
        rang.append(unique_metric_count)
        if prev_m != metric[i]:
            unique_metric_count -= 1
            prev_m = metric[i]
    #metric = [m - generation[0][0] for m in metric]
    #fv = [1 / x for x in metric]
    sum_rang = sum(rang)
    rel_rang = [x / sum_rang for x in rang]
    probs = [sum(rel_rang[:i+1]) for i in range(len(rel_rang))]
    # Draw new population
    sols = []
    #print(probs)
    for _ in range(n):
        r = random.random()
        for (i, individual) in enumerate(generation):
            if r <= probs[i]:
                sols.append(individual)
                break
    return sols

In [61]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    for j in range(len(new_generation)):
        if (len(new_generation[j]) == 2):
            new_generation[j] = new_generation[j] + (0,)
        else:
            new_generation[j] = new_generation[j][:2] + (0,)
    new_generation.sort(key=lambda t: t[0]) 
    childs = []
    for g in new_generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 1))
    for g in new_generation:
        new_solution = merge_max_and_min_charged(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 2))    
    childs.sort(key=lambda t: t[0]) 
    
    #childs = childs[:generation_size]
    
    #скрещиваем лучшие 6 решений между собой произвольным образом
    for _ in range(6):        
        #print(len(random.choice(generation[:6])[1]))
        new_solutions = multi_random_point_crossover(random.choice(new_generation[:6])[1],
                                                     random.choice(new_generation[:6])[1], 
                                                     4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 3))
    childs.sort(key=lambda t: t[0]) 
        
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными решениями
    for _ in range(6):      
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        best_solution2 = max(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, best_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 4))
    
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными худшими решениями
    for _ in range(6):
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        worst_solution2 = min(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, worst_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 5))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 3) - 6]    
    
    #селекция потомков, вероятность выбора которых обратно пропорциональна их метрике
    selected_childs = roulete_wheel_selection1(childs, (int)(generation_size / 3))
    generation.extend(selected_childs)
                
    for sol in mutated_solutions:
        generation.append(sol + (6,))
                
    generation.sort(key=lambda t: t[0])
    
    childs_count = np.array([[0, 0, 0, 0, 0, 0, 0]])
    for sol in generation:
        if sol[2] == 0:
            childs_count[0][0] += 1
        elif sol[2] == 1:
            childs_count[0][1] += 1
        elif sol[2] == 2:
            childs_count[0][2]+=1
        elif sol[2] == 3:
            childs_count[0][3]+=1
        elif sol[2] == 4:
            childs_count[0][4]+=1
        elif sol[2] == 5:
            childs_count[0][5]+=1
        else:
            childs_count[0][6]+=1
    tb = pd.DataFrame(childs_count, columns = ['0', '1', '2', '3', '4', '5', '6'])
    #display(tb)
    print([x[2] for x in generation[:20]])
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
child_selection_metric1 = generation[0][0]
print('best solution', child_selection_metric1)
labeled_res[8] = child_selection_metric1
tb.head(20)

[2, 1, 2, 2, 6, 0, 6, 6, 0, 2, 6, 1, 0, 0, 0, 0, 0, 0, 6, 6]
[13077169246.16, 14038535971.540001, 14155327025.720001, 14876448701.860001, 15276052648.58]
[2, 6, 0, 6, 1, 6, 0, 6, 0, 2, 2, 0, 0, 0, 0, 0, 3, 5, 6, 6]
[2, 6, 0, 6, 6, 6, 0, 0, 0, 3, 0, 0, 3, 0, 0, 5, 2, 4, 6, 6]
[1, 6, 6, 0, 1, 6, 6, 0, 0, 0, 0, 0, 0, 0, 2, 3, 2, 2, 6, 6]
[6, 0, 6, 6, 6, 0, 0, 3, 0, 3, 0, 0, 0, 0, 2, 2, 3, 1, 6, 6]
[1, 6, 1, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 1, 4, 4, 1, 6, 6]
[2, 1, 6, 0, 6, 1, 6, 4, 0, 0, 6, 0, 0, 0, 0, 0, 3, 1, 6, 6]
[2, 6, 0, 6, 6, 0, 6, 0, 0, 0, 0, 0, 0, 5, 5, 4, 2, 2, 6, 6]
[6, 6, 0, 2, 1, 6, 6, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 2, 6, 6]
[6, 3, 0, 6, 6, 6, 0, 0, 0, 2, 3, 3, 0, 0, 0, 0, 2, 1, 6, 6]
[6, 6, 6, 0, 6, 0, 0, 0, 3, 0, 0, 1, 0, 1, 0, 3, 2, 4, 6, 6]
[8191970972.5600004, 8251893918.54, 8287591154.1599998, 8289433642.5600004, 8349031046.1599998]
[6, 6, 0, 6, 0, 6, 0, 0, 3, 0, 0, 0, 0, 3, 3, 5, 1, 2, 6, 6]
[6, 6, 6, 0, 0, 6, 0, 0, 0, 0, 0, 0, 5, 5, 2, 2, 4, 2, 6, 6]
[6, 6, 6, 0, 3, 0,

,0,1,2,3,4,5,6
0,8,1,1,1,2,1,6


In [62]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
5453138167.78
5453138167.78


In [63]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 8:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
4 3519173551.1
5 3978039006.72
6 3197990013.22
7 3274088830.24
8 5453138167.78
result became better = False


Селекция потомков, имеющих наименьшее значение метрики приносит лучшие результаты

In [70]:
generation = copy.deepcopy(initial_generation[:])
for i in range(50):
    new_generation = copy.deepcopy(generation)
    for j in range(len(new_generation)):
        if (len(new_generation[j]) == 2):
            new_generation[j] = new_generation[j] + (0,)
        else:
            new_generation[j] = new_generation[j][:2] + (0,)
    new_generation.sort(key=lambda t: t[0]) 
    childs = []
    for g in new_generation:
        new_solution = merge_greedly_by_time(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 1))
    for g in new_generation:
        new_solution = merge_max_and_min_charged(copy.deepcopy(g[1]), 20)
        childs.append((compute_all_metrics(new_solution), new_solution, 2))    
    childs.sort(key=lambda t: t[0]) 
        
    #скрещиваем лучшие 6 решений между собой произвольным образом
    for _ in range(6):        
        #print(len(random.choice(generation[:6])[1]))
        new_solutions = multi_random_point_crossover(random.choice(new_generation[:6])[1],
                                                     random.choice(new_generation[:6])[1], 
                                                     4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 3))
    childs.sort(key=lambda t: t[0]) 
        
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными решениями
    for _ in range(6):      
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        best_solution2 = max(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, best_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 4))
    
    #берем 6 произвольных решений, выбираем из них лучшее и скрещиваем с 6
    #аналогично выбранными худшими решениями
    for _ in range(6):
        cand_solutions1 = []
        cand_solutions2 = []
        for k in range(6):
            cand_solutions1.append(random.choice(new_generation))
            cand_solutions2.append(random.choice(new_generation))
        best_solution1 = max(cand_solutions1, key=lambda x: x[0])[1]
        worst_solution2 = min(cand_solutions2, key=lambda x: x[0])[1]
        new_solutions = multi_random_point_crossover(best_solution1, worst_solution2, 4)
        for new_solution in new_solutions:
            childs.append((compute_all_metrics(new_solution), new_solution, 5))
    childs.sort(key=lambda t: t[0])
    
    mutated_solutions = []
    #произвольная мутация над 2 лучшими решениями
    mutated_solutions.extend(get_random_mutation(copy.deepcopy(new_generation[:2])))
    #произвольная мутация над 2 худшими решениями
    #mutated_solutions.extend(get_random_mutation(new_generation[-2:]))
    #распределяющая мутация над 2 лучшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[:2])))
    #распределяющая мутация над 2 худшими решениями
    mutated_solutions.extend(get_distributing_mutation(copy.deepcopy(new_generation[-2:])))
    
    generation = new_generation[:generation_size - (int)(generation_size / 4) - 6]    
    
    generation.extend(childs[:(int)(generation_size / 4)])
                
    for sol in mutated_solutions:
        generation.append(sol + (6,))
                
    generation.sort(key=lambda t: t[0])
    
    childs_count = np.array([[0, 0, 0, 0, 0, 0, 0]])
    for sol in generation:
        if sol[2] == 0:
            childs_count[0][0] += 1
        elif sol[2] == 1:
            childs_count[0][1] += 1
        elif sol[2] == 2:
            childs_count[0][2]+=1
        elif sol[2] == 3:
            childs_count[0][3]+=1
        elif sol[2] == 4:
            childs_count[0][4]+=1
        elif sol[2] == 5:
            childs_count[0][5]+=1
        else:
            childs_count[0][6]+=1
    tb = pd.DataFrame(childs_count, columns = ['0', '1', '2', '3', '4', '5', '6'])
    #display(tb)
    #print([x[2] for x in generation[:20]])
    if i % 10 == 0:
        print([s[0] for s in generation][:5])
child_selection_metric1 = generation[0][0]
print('best solution', child_selection_metric1)
labeled_res[9] = child_selection_metric1
tb.head(20)

[12194629510.02, 12270014731.300001, 12465030820.1, 12521524297.559999, 13077169246.16]
[4774564363.7600002, 5277417702.04, 5325802073.8199997, 5393250856.3599997, 5399109804.1400003]
[3394771116.2200003, 3403415866.5999999, 3415276917.54, 3426683820.2200003, 3433308014.5999999]
[2704830566.6799998, 2727374626.5599999, 2730056377.4400001, 2731590816.4400001, 2733238076.6799998]
[2459191059.5799999, 2460869437.5799999, 2469615638.1399999, 2471392424.2000003, 2480909719.3600001]
best solution 2244374983.82


,0,1,2,3,4,5,6
0,9,0,0,4,0,1,6


In [71]:
is_error = False
for sol in generation:
    if sol[0] != compute_all_metrics(sol[1]):
        is_error = True
print('error =', is_error)
sol1 = generation[0]
print(sol1[0])
print(compute_all_metrics(sol1[1]))

error = False
2244374983.82
2244374983.82


In [72]:
for key, value in labeled_res.items():
    print(key, value)
if min(labeled_res, key=labeled_res.get) == 9:
    print('result became better =', True)
    best_generation = generation
else:
    print('result became better =', False)

0 15375324356.6
1 4803355451.4
2 8643682071.86
3 3208763123.02
4 3519173551.1
5 3978039006.72
6 3197990013.22
7 3274088830.24
8 5453138167.78
9 2244374983.82
result became better = True


---

#### Сохранение распределения

In [73]:
def save_solution(file_name, solution):
    data = []
    for agent in solution:
        for act in solution[agent]:
            data.append([agent, act])
    pd.DataFrame(data, columns=['agent_id', 'act_id']).to_csv(file_name, index=False)

In [74]:
save_solution('res.csv', best_generation[0][1])